Boston Housing Dataset Test
===========================

In this notebook we test revrand's ARD basis functions on the Boston housing dataset.

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.cross_validation import KFold

from revrand import StandardLinearModel
import revrand.basis_functions as bf
from revrand.metrics import smse, msll
from revrand.btypes import Parameter, Positive, Bound

# Log output to the terminal attached to this notebook
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")


In [2]:
# Load the data
boston = load_boston()
X = boston.data
y = boston.target - boston.target.mean()

N, D = X.shape
lenscale = 10.
nbases = 200
lenARD = lenscale * np.ones(D)


In [3]:
# Construct basis functions
base = bf.RandomMatern32(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive())) + \
    bf.LinearBasis(onescol=True)
#base = bf.spectralmixture(Xdim=D, bases_per_component=5, lenscales_init=[Parameter(l, Positive()) for l in lenARD]) + \
#    bf.LinearBasis(onescol=True)
#base = bf.SigmoidalBasis(centres=X[np.random.choice(N, 10), :], lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
#base = bf.FastFoodRBF(Xdim=D, nbases=nbases, lenscale_init=Parameter(lenARD, Positive())) + \
#    bf.LinearBasis(onescol=True)
    

In [4]:
# Cross val
folds = 5
av_smse, av_msll = 0., 0.
av_smse_gp, av_msll_gp = 0., 0

slm = StandardLinearModel(base)

for i, (tr_ind, ts_ind) in enumerate(KFold(len(y), n_folds=folds, shuffle=True)):
    
    # Training
    slm.fit(X[tr_ind], y[tr_ind])
    
    # Prediction
    Ey, Vf, Vy = slm.predict_moments(X[ts_ind])

    # Validation
    f_smse, f_msll = smse(y[ts_ind], Ey), msll(y[ts_ind], Ey, Vy, y[tr_ind])

    av_smse += f_smse
    av_msll += f_msll
    
    print("Fold: {},\n\trevrand: SMSE = {}, MSLL = {}".format(i, f_smse, f_msll))
    
av_smse /= folds
av_msll /= folds


Fold: 0,
	revrand: SMSE = 0.19151829247906846, MSLL = -0.8444097880080966
Fold: 1,
	revrand: SMSE = 0.2539768410760672, MSLL = -0.7052400360824306
Fold: 2,
	revrand: SMSE = 0.25730056361000564, MSLL = -0.717408313130109
Fold: 3,
	revrand: SMSE = 0.2624221337658452, MSLL = -0.1807631527854384
Fold: 4,
	revrand: SMSE = 0.2034590580297917, MSLL = -0.835076876944985


In [5]:
# Print results
print("Final:\n\trevrand: SMSE = {}, MSLL = {}".format(av_smse, av_msll))


Final:
	revrand: SMSE = 0.23373537779215567, MSLL = -0.6565796333902119
